In [ ]:
import numpy as np
import sympy as sp
from sympy import symbols, diff

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from skimage import measure

from symbolic_hulls import *

### Parabolic Functions in 2D

In [ ]:
x, y = symbols('x y')
xp, yp = symbols('x_p y_p')

In [ ]:
f1 = x**2 + 3*x - 2
f2 = 3*x**2 - 4*x - 5

proj, variables, pvariables = projection_function(f1, f2)
disc1 = recursive_discriminant(proj, variables)
sols1 = [sp.N(sol) for sol in sp.solve(disc1, pvariables)]

proj, variables, pvariables = projection_function(f2, f1)
disc2 = recursive_discriminant(proj, variables)
sols2 = [sp.N(sol) for sol in sp.solve(disc2, pvariables)]

In [ ]:
x_space = np.linspace(-3, 7, 100)
f1_vals = [f1.subs(x, val) for val in x_space]
f2_vals = [f2.subs(x, val) for val in x_space]

plt.plot(x_space, f1_vals, label='f1')
plt.plot(x_space, f2_vals, label='f2')

plt.plot(sols1, [f1.subs(x, sol) for sol in sols1], 'o', color='blue', label='f1 Tangent Points')
plt.plot(sols2, [f2.subs(x, sol) for sol in sols2], 'o', color='orange', label='f2 Tangent Points')
plt.legend()
plt.show()

### Higher Degree Functions in 3D

In [ ]:
f1 = x**4 - x**2 + 2*y**2 + 1
f2 = x**2 + y**2 + (1/2)

proj, variables, pvariables = projection_function(f1, f2)
disc1 = recursive_discriminant(proj, (x, y))
display(disc1)

proj, variables, pvariables = projection_function(f2, f1)
disc2 = recursive_discriminant(proj, (x, y))
display(disc2)

In [ ]:
# Convert SymPy expressions to numerical functions
f1_func = sp.lambdify((x, y), f1, 'numpy')
f2_func = sp.lambdify((x, y), f2, 'numpy')

disc1_func = sp.lambdify((xp, yp), disc1, 'numpy')
disc2_func = sp.lambdify((xp, yp), disc2, 'numpy')

# Define grid for plotting
x_vals = np.linspace(-1.7, 1.7, 1000)
y_vals = np.linspace(-1.7, 1.7, 1000)
X, Y = np.meshgrid(x_vals, y_vals)

# Compute function values
F1_vals = f1_func(X, Y)
F2_vals = f2_func(X, Y)
Disc1_vals = disc1_func(X, Y)
Disc2_vals = disc2_func(X, Y)

# Plot the surfaces
surface_f1 = go.Surface(x=X, y=Y, z=F1_vals, colorscale=[[0, "blue"], [1, "blue"]], name="f1 Surface", showscale=False)
surface_f2 = go.Surface(x=X, y=Y, z=F2_vals, colorscale=[[0, "orange"], [1, "orange"]], name="f2 Surface", showscale=False)
data = [surface_f1, surface_f2]

# Plot the 0-level contours of the discriminants
disc1_contours = measure.find_contours(Disc1_vals, level=0)
disc2_contours = measure.find_contours(Disc2_vals, level=0)
for contour in disc1_contours:
    x_indices = contour[:, 1]
    y_indices = contour[:, 0]

    x_real = x_vals[(x_indices).astype(int)]
    y_real = y_vals[(y_indices).astype(int)]

    contour = go.Scatter3d(
        x=x_real, 
        y=y_real, 
        z=f1_func(x_real, y_real),
        mode='lines',
        line=dict(color='black', width=3),
    )

    data.append(contour)

for contour in disc2_contours:
    x_indices = contour[:, 1]
    y_indices = contour[:, 0]

    x_real = x_vals[(x_indices).astype(int)]
    y_real = y_vals[(y_indices).astype(int)]

    contour = go.Scatter3d(
        x=x_real, 
        y=y_real, 
        z=f2_func(x_real, y_real),
        mode='lines',
        line=dict(color='black', width=3)
    )

    data.append(contour)

# Combine all plots
fig = go.Figure(data=data)

# Layout configuration
fig.update_layout(
    title="3D Plot of f1 and f2 with 0-Contour Levels of Discriminants",
    width=800,
    height=800,
    scene=dict(
        xaxis_title="X-axis",
        yaxis_title="Y-axis",
        zaxis_title="Function Value",
        xaxis=dict(range=[-2, 2]),
        yaxis=dict(range=[-2, 2]),
        zaxis=dict(range=[-1, 3]),
        aspectmode='cube',
    )
)

fig.update_layout(showlegend=False)

# Show the plot
fig.show()